In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_squared_error,mean_absolute_error,r2_score


In [3]:
# %pwd 
os.chdir("../")
# %pwd

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path


In [ ]:
from Wine_Quality_Prediction.constants import *
from Wine_Quality_Prediction.utils.common import read_yaml,create_directories

In [6]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
        )

        return data_ingestion_config

In [7]:
from Wine_Quality_Prediction import logger
from Wine_Quality_Prediction.utils.common import get_size
import urllib.request as request

In [8]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")

In [9]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
except Exception as e:
    raise e

[2025-10-18 00:29:09,700: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-10-18 00:29:09,703: INFO: common: yaml file: params.yaml loaded successfully]
[2025-10-18 00:29:09,707: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-10-18 00:29:09,709: INFO: common: created directory at: artifacts]
[2025-10-18 00:29:09,711: INFO: common: created directory at: artifacts/data_ingestion]
[2025-10-18 00:29:10,478: INFO: 739313186: artifacts/data_ingestion/winequality-red.csv download! with following info: 
Date: Fri, 17 Oct 2025 23:29:06 GMT
Content-Type: text/html; charset=utf-8
Vary: X-PJAX, X-PJAX-Container, Turbo-Visit, Turbo-Frame, X-Requested-With,Accept-Encoding, Accept, X-Requested-With
x-repository-download: git clone https://github.com/singhmanpreet1308/End_to_End_ML_Pipeline_Wine_Quality.git
x-raw-download: https://raw.githubusercontent.com/singhmanpreet1308/End_to_End_ML_Pipeline_Wine_Quality/main/data/winequality-red.csv
ETag: W/"7027c116f238e15